In [ ]:
# Check if the protected prefixes correlated with A records of Tranco 1M list in 2024 jan 01.

In [ ]:
# Find all the protected prefixes of individual scrubbers
import pandas as pd
import csv


scrubber_asns = ["32787", "13335", "19551", "198949", "19905"]
year = "2017"

path = "/home/shyam/jupy/ddos_scrubber/data/"
all_record_prefixes = [] # Stores number of customer prefixes for a month


for scrubber_asn in scrubber_asns:
    single_record_prefixes = [] # Stores number of customer prefixes for a month

    confirmed_customers1 = pd.read_csv(path + "confirmed_customers_as"+scrubber_asn+"_" +year+".csv")
    confirmed_customers1_unique_origin_prefixes = confirmed_customers1['prefix'].unique()
    
    if scrubber_asn not in ["13335"]: # Cloudflare does not have path prepending case
        path_prepending = pd.read_csv(path + "unique_optimized_provider_as"+scrubber_asn+"_path_prepend_01_jan_"+year+".csv")
        path_prepending_unique_origin_prefixes = path_prepending['prefix'].unique()
        confirmed_customers2_prefixes = list(set(path_prepending_unique_origin_prefixes)-set(confirmed_customers1_unique_origin_prefixes))
    else:
        confirmed_customers2_prefixes = []
        
    if  scrubber_asn not in ["13335"]: # Cloudflare does not have path prepending case
        df = pd.read_csv(path + "unique_optimized_provider_not_as"+scrubber_asn+"_01_jan_"+year+"_v3.csv")
        sibling_path = df.loc[(df['siblings'] == 1) & (df['new_provider_sibling_check'] == int(scrubber_asn))]
        sibling_path_unique_origin_prefixes = sibling_path['prefix'].unique()
    else:
        sibling_path_unique_origin_prefixes = []

        
    confirmed_customers3_prefixes = list(set(sibling_path_unique_origin_prefixes)-set(confirmed_customers2_prefixes)-set(confirmed_customers1_unique_origin_prefixes))

    confirmed_customers_prefixes = list(confirmed_customers1_unique_origin_prefixes) + list(confirmed_customers2_prefixes) + list(confirmed_customers3_prefixes)
    
    # Save to CSV file
    with open("../data/customers_prefixes_scrubber_"+scrubber_asn+"_"+year+".csv", "w", newline="") as file:
#     with open("../data/customers_prefixes_scrubber_"+scrubber_asn+"_2024_cloudflare.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["prefix"])  # Write header
        for prefix in confirmed_customers_prefixes:
            writer.writerow([prefix])  # Write each name in a new row
    print("Done for scrubber ", scrubber_asn)
print("Completed.")

In [ ]:
# Find total protected prefixes of all scrubbers
import os
import re
import pandas as pd

year = "2017"

pattern = r"^customers_prefixes_scrubber_\d+_"+year+"\.csv$"
directory = "/home/shyam/jupy/ddos_scrubber/data"

csv_files = [f for f in os.listdir(directory) if re.match(pattern, f)]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append it to the list
for file in csv_files:
    df = pd.read_csv(directory+"/"+file)
    dataframes.append(df)

# Merge all dataframes into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe into a new CSV file
merged_df.to_csv("../data/customers_prefixes_scrubber_all_"+year+".csv", index=False)  

merged_df_unique = merged_df.drop_duplicates()

# Save the merged dataframe into a new CSV file
merged_df_unique.to_csv("../data/customers_prefixes_unique_scrubber_all_"+year+".csv", index=False) 

print("Completed. All the prefixes of %s saved in file customers_prefixes_scrubber_all_%s.csv"  %(year, year))

In [1]:
# Find total protected prefixes of all scrubbers monthwise
# After TMA
import os
import re
import pandas as pd

years = ["2020", "2021", "2022", "2023", "2024"]
day = "01"
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# List your scrubber ASNs here
scrubber_asns = ["32787", "13335", "19551", "198949", "19905"]

for year in years:
    
    for mon in months:
        print(f"🔄 Processing month: {mon}")

        all_matching_files = []

        for scrubber in scrubber_asns:
            base_dir = f"../data/after_tma/as{scrubber}"

            # Regex to match files like customers_prefixes_scrubber_13335_01_jan_2024.csv
            pattern = rf"^customers_prefixes_scrubber_\d+_{day}_{mon}_{year}\.csv$"

            for root, dirs, files in os.walk(base_dir):
                for file in files:
                    if re.match(pattern, file):
                        full_path = os.path.join(root, file)
                        all_matching_files.append(full_path)

        # Merge all matching files
        if all_matching_files:
            dataframes = [pd.read_csv(f) for f in all_matching_files]
            merged_df = pd.concat(dataframes, ignore_index=True)

            print(f"✅ Without removing duplicates: {len(merged_df)}")
            # Optional: Drop duplicates
            merged_df_unique = merged_df.drop_duplicates()

            # Output path
            output_file = f"../data/after_tma/customers_prefixes_scrubber_all_{day}_{mon}_{year}.csv"
            merged_df_unique.to_csv(output_file, index=False)
            print(f"📁 Saved merged file: {output_file} ({len(merged_df_unique)} unique rows)")
        else:
            print(f"⚠️ No files found for {mon} {year}")

    print(f"✅ Done for all months for year .{year}")

🔄 Processing month: jan
✅ Without removing duplicates: 3154
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_jan_2020.csv (3153 unique rows)
🔄 Processing month: feb
✅ Without removing duplicates: 3384
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_feb_2020.csv (3383 unique rows)
🔄 Processing month: mar
✅ Without removing duplicates: 3551
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_mar_2020.csv (3550 unique rows)
🔄 Processing month: apr
✅ Without removing duplicates: 3295
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_apr_2020.csv (3292 unique rows)
🔄 Processing month: may
✅ Without removing duplicates: 3414
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_may_2020.csv (3411 unique rows)
🔄 Processing month: jun
✅ Without removing duplicates: 3506
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_jun_2020.csv (3503 unique rows)
🔄 Pr

✅ Without removing duplicates: 12514
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_sep_2024.csv (12106 unique rows)
🔄 Processing month: oct
✅ Without removing duplicates: 12740
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_oct_2024.csv (12337 unique rows)
🔄 Processing month: nov
✅ Without removing duplicates: 12845
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_nov_2024.csv (12448 unique rows)
🔄 Processing month: dec
✅ Without removing duplicates: 12713
📁 Saved merged file: ../data/after_tma/customers_prefixes_scrubber_all_01_dec_2024.csv (12362 unique rows)
✅ Done for all months for year .2024


In [2]:
# Find the number of IP addreses in the protected prefix list
import pandas as pd
import ipaddress
year = "2017"

def count_ips_in_prefix(prefix):
    """Calculate the total number of IP addresses in a given prefix."""
    return 2 ** (ipaddress.ip_network(prefix, strict=False).max_prefixlen - ipaddress.ip_network(prefix).prefixlen)

# Load the CSV file
csv_file = "../data/customers_prefixes_unique_scrubber_all_"+year+".csv"  # Update this with your actual file path
df = pd.read_csv(csv_file)

# Assuming the column containing prefixes is named 'prefix'
total_ips = df['prefix'].apply(count_ips_in_prefix).sum()

print(f"Total number of IP addresses: {total_ips}")

Total number of IP addresses: 10880332376531662573245952


In [11]:
# Check how many prefixes contain super prefix
from ipaddress import ip_network
import pandas as pd
year = "2024"

def process_networks(networks):
    superprefixes = set()
    independent_prefixes = set(networks)

    for i, net in enumerate(networks):
        for j in range(i):  # Only check prefixes before the current one (less specific ones)
            if networks[j].supernet_of(net):
                superprefixes.add(networks[j])
                independent_prefixes.discard(net)

    return superprefixes, independent_prefixes

def find_superprefixes_and_independent(prefix_list):
    # Convert strings to ip_network objects
    networks = [ip_network(p) for p in prefix_list]
    
    # Separate IPv4 and IPv6 networks
    ipv4_networks = sorted([n for n in networks if n.version == 4], key=lambda net: net.prefixlen)
    ipv6_networks = sorted([n for n in networks if n.version == 6], key=lambda net: net.prefixlen)
    
    superprefixes_v4, independent_v4 = process_networks(ipv4_networks)
    superprefixes_v6, independent_v6 = process_networks(ipv6_networks)
    
    return superprefixes_v4 | superprefixes_v6, independent_v4 | independent_v6 # Union merge of the results

# Example usage
# prefixes = [
#     "192.168.0.0/16",
#     "192.168.1.0/24",
#     "2001:db8::/32",
#     "2001:db8:1::/48",
#     "10.0.0.0/8",
#     "10.1.0.0/16",
#     "202.70.64.0/24",
#     "2001:4860::/32",
#     "2001:4860:4860::8888/128"
# ]

df = pd.read_csv("../data/customers_prefixes_unique_scrubber_all_"+year+".csv")
prefixes = df["prefix"]

superprefixes, independent_prefixes = find_superprefixes_and_independent(prefixes)
# print("Superprefixes:", superprefixes)
# print("Independent prefixes:", independent_prefixes)
print("Count of Independent prefixes:", len(independent_prefixes))


Count of Independent prefixes: 10267


In [ ]:
# Compare the ip addresses with the protected prefixes using pytricia loop
import pandas as pd
import ipaddress
import pytricia


def build_prefix_trees(prefix_list):
    """Builds two Patricia Tries: one for IPv4 and one for IPv6."""
    pt_v4 = pytricia.PyTricia()
    pt_v6 = pytricia.PyTricia()

    for prefix in prefix_list:
        network = ipaddress.ip_network(prefix)
        if network.version == 4:

            pt_v4[prefix] = True
        else:
            pt_v6[prefix] = True

    return pt_v4, pt_v6

def count_covered_ips(ip_list, pt_v4, pt_v6):
    """Counts how many IPs are covered by the prefixes in the Patricia Tries."""
    count = 0
    for ip in ip_list:
        ip_obj = ipaddress.ip_address(ip)
        if ip_obj.version == 4 and pt_v4.get(ip):
            count += 1
        elif ip_obj.version == 6 and pt_v6.get(ip):
            count += 1

    return count


def get_covered_ips(ip_list, pt_v4, pt_v6):
    """Returns the list of IPs covered by prefixes in the Patricia Trie."""
    covered_ips = []
    for ip in ip_list:
        ip_obj = ipaddress.ip_address(ip)
        if ip_obj.version == 4 and pt_v4.get(ip):
            covered_ips.append(ip)
        elif ip_obj.version == 6 and pt_v6.get(ip):
            covered_ips.append(ip)
#     covered_ips = [ip for ip in ip_list if prefix_tree.get(ip)]
    return covered_ips



# Protected prefixes lists using all five scrubberscovered_count
df = pd.read_csv("../data/customers_prefixes_scrubber_all_"+year+".csv") 
protected_prefixes = df["prefix"].tolist()

# Get A record of tranco list of 1M domains
# tranco_domain_ip_list = []

tranco_ip_addresses = []

# Get matching domain lists
# tranco_domains = []

with open("../data/tranco-1m/a_records_tranco.txt", "r") as infile:
    for line in infile:
        tranco_dict = {}
        
        if " A " in line:
            parts = line.split()
            if len(parts) >= 3:
                ip = parts[2]
                
                tranco_ip_addresses.append(parts[2])
                domain = parts[0].rstrip('.')
                tranco_dict[ip] = domain
#                 tranco_domains.append(domain)
#     tranco_domain_ip_list.append(tranco_dict)
    

# Build separate prefix trees for IPv4 and IPv6
pt_v4, pt_v6 = build_prefix_trees(protected_prefixes)

# Find the number of IPs covered
covered_count = count_covered_ips(tranco_ip_addresses, pt_v4, pt_v6)

print(f"Number of IPs covered: {covered_count}")

# Find the IPs covered
covered_ips = get_covered_ips(tranco_ip_addresses, pt_v4, pt_v6)

unique_covered_ips = list(set(covered_ips))
print(f"Number of unique IPs covered: {len(unique_covered_ips)}")

# Save covered ips in a .txt file.
with open("../data/tranco-1m/tranco_scrubber_covered_ip.txt", "w", encoding="utf-8") as f:
    for ip in unique_covered_ips:
        f.write(f"{ip}\n")
print("Completed.")       


In [ ]:
# NOT USED FROM HERE, We used OpenIntel instead
# Find ranks of tranco 1M domains with their A records (IP address/es) and save it into a file.
import csv

# Load domains and assign ranks
rank_dict = {}
with open("../data/tranco-1m/top-1m.txt", "r", encoding="utf-8") as f:
    for rank, domain in enumerate(f, start=1):  # Auto-assign rank
        domain = domain.strip()
        if domain:
            rank_dict[domain] = str(rank)

# Parse massdns results
resolved = {}
with open("../data/tranco-1m/a_records_tranco.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 3 and parts[1] == "A":
            domain = parts[0].rstrip('.')
            ip = parts[2]
            if domain not in resolved:
                resolved[domain] = []
            resolved[domain].append(ip) # Same domain can  have multiple ip addresses

# Write final results with rank
with open("../data/tranco-1m/tranco_resolved_with_rank.csv", "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Rank", "Domain", "IPs"])
    for domain, ips in resolved.items():
        writer.writerow([rank_dict.get(domain, "N/A"), domain, ",".join(ips)])

print("Resolution completed: Check tranco_resolved_with_rank.csv")


In [ ]:
# Find rank, domain names of the covered ip addresses.
import csv
import io
import pandas as pd

# List of target IP addresses
tranco_scrubber_covered_ips = []

with open("../data/tranco-1m/tranco_scrubber_covered_ip.txt", "r", encoding="utf-8") as f:
    for line in f:
        tranco_scrubber_covered_ips.append(line.strip())


target_ips = set(tranco_scrubber_covered_ips)

# Sample CSV data (as a string for demonstration)

# csv_data = """Rank,Domain,IPs
# 1,google.com,142.250.74.78,172.217.14.206
# 2,youtube.com,172.217.14.206
# 3,example.com,8.8.8.8
# 4,another-example.com,8.8.8.8
# """

# Load the CSV into a dictionary {IP -> [(Rank, Domain), (Rank, Domain), ...]}
ip_to_rank = {}

# Read the CSV object
csv_file = io.StringIO("../data/tranco-1m/tranco_resolved_with_rank.csv")
reader = csv.reader(csv_file)


with open('../data/tranco-1m/tranco_resolved_with_rank.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header
    
    for row in csvFile:
        rank, domain, *ips = row  # Extract rank, domain, and IPs
        for ip in ips:
            ip = ip.strip()
            if ip:
                if ip not in ip_to_rank:
                    ip_to_rank[ip] = []
                ip_to_rank[ip].append((rank, domain))  # Store multiple (Rank, Domain) pairs



# Find matching domains and ranks
results = []
for ip in target_ips:
    if ip in ip_to_rank:
        ranks, domains = zip(*ip_to_rank[ip])  # Unpack into separate lists
        results.append((ip, ";".join(ranks), ";".join(domains)))  # Store all matches
    else:
        results.append((ip, "N/A", "N/A"))  # No match found

# Save results to a CSV file
with open("../data/tranco-1m/ip_domains_ranks.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["IP", "Rank(s)", "Domain(s)"])  # Write header
    
    # Write the results rows
    for row in results:
        writer.writerow(row)

print("Results saved to 'ip_domains_ranks.csv'.")


In [ ]:
# Get ranks of the domains
import csv

# List to store ranks
ranks = []

# Read the CSV file to extract ranks
with open("../data/tranco-1m/ip_domains_ranks.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    for row in reader:
        # row[1] contains Rank(s), which is a semicolon-separated string
        if row[1] != "N/A":
            ranks.extend(row[1].split(";"))  # Add each rank to the list

# Convert ranks to integers (if needed)
ranks = [int(rank) for rank in ranks]

# Print the ranks list
print("No. of domains covered by 7985 IP addresses are %s" %len(ranks))


In [ ]:
len(ranks)

In [ ]:
# Find the number of domains hosted by those IP addresses
# Reverse IP lookup, results saved in /home/shyam/jupy/ddos_scrubber/data/tranco-1m/ttranco_scrubber_covered_ptr_results.txt

from ipaddress import ip_address

# Input and output file names
ip_file = "../data/tranco-1m/tranco_scrubber_covered_ip.txt"
ptr_file = "../data/tranco-1m/tranco_scrubber_covered_ptr.txt"

# Step 1: Convert IPs to PTR queries
def ip_to_ptr(ip):
    """Convert an IP address to a PTR record format."""
    reversed_ip = ip_address(ip).reverse_pointer
    return reversed_ip
    
# Step 2: Write converted PTR into a file
with open(ip_file, "r") as f, open(ptr_file, "w") as out:
    for line in f:
        ip = line.strip()
        if ip:
            out.write(ip_to_ptr(ip) + "\n")

# Run massdns
# massdns -r lists/resolvers.txt -t PTR -o S -w /home/shyam/jupy/ddos_scrubber/data/tranco-1m/tranco_scrubber_covered_ptr_results.txt /home/shyam/jupy/ddos_scrubber/data/tranco-1m/tranco_scrubber_covered_ptr.txt
# 12769 domains found out of 7985 IP addresses protected. 

In [ ]:
# Tranco 1M list of 01 Jan 2024 showed 14449 domains, while reverse IP lookup on 11 Feb 2025 showed 12769 domains.
# Compare these two domain names to find number of same domains and different domains
import csv

# List to store domains from Tranco 1M list
domains_tranco = []

# Read the CSV file to extract ranks
with open("../data/tranco-1m/ip_domains_ranks.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    for row in reader:
        # row[2] contains Domain(s), which is a semicolon-separated string
        if row[2] != "N/A":
            domains_tranco.extend(row[2].rsplit(";"))  # Add each domain to the list

# Convert ranks to integers (if needed)
domains_tranco = [domain for domain in domains_tranco]

# Get domains from reverse IP lookup of 79885 unique IP addrsses on 11 Feb 2025
domains_reverse_lookup = []
with open("../data/tranco-1m/tranco_scrubber_covered_ptr_results.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
#         if len(parts) >= 3 and parts[1] == "PTR":
        domain = parts[2].rstrip('.')
        domains_reverse_lookup.append(domain) 

# Find unique domains
unique_domains_reverse_lookup = list(set(domains_reverse_lookup))
unique_domains_tranco = list(set(domains_tranco))

print("No. of domains in Tranco 1M list covered by 7985 IP addresses are %s and unique number of domains are %s.\n" %(len(domains_tranco), len(unique_domains_tranco)) )

print("No. of domains in reverse IP lookup covered by 7985 IP addresses are %s and unique number of domains are %s.\n" %(len(domains_reverse_lookup),(len(unique_domains_reverse_lookup))) )

# Similar domains in Tranco 1M list and reverse domain list
same_domains = list(set(domains_tranco) & set(unique_domains_reverse_lookup))
print("No. of same domains in Tranco 1M list and reverse domain list is %s.\n" %len(same_domains))

domains_tranco_not_reverse = list(set(domains_tranco) - set(domains_reverse_lookup))
print("No. of domains in Tranco 1M list but not in reverse domain list is %s.\n" %len(domains_tranco_not_reverse))

domains_reverse_not_tranco = list(set(domains_reverse_lookup) - set(domains_tranco))
print("No. of domains in reverse domain list but not in Tranco 1M list is %s.\n" %len(domains_reverse_not_tranco))

total_protected_domains = len(same_domains) + len(domains_tranco_not_reverse) + len(domains_reverse_not_tranco)
print("Total number of protected domains by scrubbing is %s" %(total_protected_domains))